

**模型背景**：
- `CrossBasicUNet` 是一个基于经典 U-Net 的扩展模型，用于处理医疗影像分割等任务。
- U-Net 的核心思想是通过编码-解码结构结合跳跃连接（skip connections），高效提取特征并恢复空间信息。

- **跨层特征共享**：
  - 与传统 U-Net 不同，`CrossBasicUNet` 的每个解码器层都会共享来自所有编码器层的特征，而不仅仅是同层级的编码器特征。
  - 通过 `UpCatAll` 模块，将所有编码器层的特征调整为一致的尺寸并拼接到当前解码器层输入中。
- **灵活的维度支持**：
  - 支持 1D, 2D 和 3D 数据处理（`spatial_dims` 参数）。
- **模块化设计**：
  - 核心计算单元（`TwoConv`, `Down`, `UpCatAll`）模块化封装，便于扩展和调试。

---

### **2. 代码模块分解**

#### **(a) 核心组件**
1. **`TwoConv` 模块**：
   - 两次卷积操作，用于提取局部特征，同时保持空间尺寸不变。
   - 默认激活函数为 `LeakyReLU`，归一化方式为 `InstanceNorm`。
   
2. **`Down` 模块**：
   - 包含一次最大池化（MaxPooling）操作和 `TwoConv` 模块。
   - 实现下采样，减少空间尺寸并增加特征通道数。

3. **`UpCatAll` 模块**：
   - **功能**：
     - 将解码器的上采样特征与所有编码器层的特征进行拼接。
     - 使用插值调整各编码器层特征的尺寸，以匹配当前解码器层。
   - **拼接逻辑**：
     - 对所有编码器层特征使用 `torch.nn.functional.interpolate` 调整为解码器的目标尺寸。
     - 拼接后的总通道数为：
       \[
       \text{总通道数} = \text{上采样通道数} + \sum \text{编码器特征通道数}
       \]
   - **模块化封装**：
     - 支持不同的上采样方式（默认 `deconv`）。

---

#### **(b) 主模型 `CrossBasicUNet`**

1. **结构**：
   - 编码器部分：
     - 包含四层 `Down` 模块，逐层提取高层语义特征。
     - 每层输出的特征会保存在 `encoder_features` 中。
   - 解码器部分：
     - 使用四层 `UpCatAll` 模块。
     - 逐层恢复空间分辨率，并将所有编码器特征拼接到当前解码器层。
   - 最终通过 `1x1` 卷积输出结果。

2. **参数可配置**：
   - 支持自定义输入通道数、输出通道数、特征通道数配置（`features` 参数）。
   - 可调整激活函数、归一化方法、丢弃率、上采样方式等。

3. **`forward` 流程**：
   - 编码阶段：依次计算 `x0, x1, x2, x3, x4`，并存入 `encoder_features`。
   - 解码阶段：通过 `UpCatAll` 模块逐层融合并恢复空间尺寸，最终生成分割结果。


In [1]:
import torch
from torchsummary import summary
from cross_basic_unet import CrossBasicUNet

# 模型初始化
model = CrossBasicUNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    features=[64,64,128,256,512,64]
)

# 打印模型结构
summary(model, (1, 256, 256))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 256, 256]             320
    InstanceNorm2d-2         [-1, 32, 256, 256]              64
           Dropout-3         [-1, 32, 256, 256]               0
         LeakyReLU-4         [-1, 32, 256, 256]               0
            Conv2d-5         [-1, 32, 256, 256]           9,248
    InstanceNorm2d-6         [-1, 32, 256, 256]              64
           Dropout-7         [-1, 32, 256, 256]               0
         LeakyReLU-8         [-1, 32, 256, 256]               0
         MaxPool2d-9         [-1, 32, 128, 128]               0
           Conv2d-10         [-1, 32, 128, 128]           9,248
   InstanceNorm2d-11         [-1, 32, 128, 128]              64
          Dropout-12         [-1, 32, 128, 128]               0
        LeakyReLU-13         [-1, 32, 128, 128]               0
           Conv2d-14         [-1, 32, 1

In [2]:
# 生成输入数据
input_tensor = torch.randn(1, 1, 256, 256)  # Batch size = 1, 1 通道, 256x256 图像

# 前向传播
output = model(input_tensor)

# 检查输出形状
print("Output shape:", output.shape)


Output shape: torch.Size([1, 1, 256, 256])


In [6]:
import torch
from torchsummary import summary
from monai.networks.nets import BasicUNet
from cross_basic_unet import CrossBasicUNet  # 确保 cross_basic_unet.py 在同目录下或可导入

# 初始化 BasicUNet 和 CrossBasicUNet
basic_unet = BasicUNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    features=[64,64,128,256,512,64]
)

cross_basic_unet = CrossBasicUNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    features=[64,64,128,256,512,64]
)

# 定义输入张量的形状
input_size = (1, 256, 256)

# 打印 BasicUNet 的模型结构
print("\n=== BasicUNet ===")
summary(basic_unet, input_size)

# 打印 CrossBasicUNet 的模型结构
print("\n=== CrossBasicUNet ===")
summary(cross_basic_unet, input_size)


BasicUNet features: (64, 64, 128, 256, 512, 64).

=== BasicUNet ===
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]             640
    InstanceNorm2d-2         [-1, 64, 256, 256]             128
           Dropout-3         [-1, 64, 256, 256]               0
         LeakyReLU-4         [-1, 64, 256, 256]               0
            Conv2d-5         [-1, 64, 256, 256]          36,928
    InstanceNorm2d-6         [-1, 64, 256, 256]             128
           Dropout-7         [-1, 64, 256, 256]               0
         LeakyReLU-8         [-1, 64, 256, 256]               0
         MaxPool2d-9         [-1, 64, 128, 128]               0
           Conv2d-10         [-1, 64, 128, 128]          36,928
   InstanceNorm2d-11         [-1, 64, 128, 128]             128
          Dropout-12         [-1, 64, 128, 128]               0
        LeakyReLU-13         [-1, 6



| **模型名称**      | **总参数量 (Total params)** | **可训练参数 (Trainable params)** | **输入大小 (Input size)** | **前向/反向过程大小 (Forward/backward pass size)** | **参数大小 (Params size)** | **估计总大小 (Estimated Total Size)** |
|-------------------|----------------------------|-----------------------------------|--------------------------|--------------------------------------------------|----------------------------|---------------------------------------|
| **BasicUNet**     | 7,903,361                 | 7,903,361                        | 0.25 MB                 | 848.00 MB                                       | 30.15 MB                  | 878.40 MB                            |
| **CrossBasicUNet**| 11,810,945                | 11,810,945                       | 0.25 MB                 | 848.00 MB                                       | 45.06 MB                  | 893.31 MB                            |


### **对比分析**

1. **参数量增加**：
   - `CrossBasicUNet` 的总参数量为 `11,810,945`，相比 `BasicUNet` 的 `7,903,361`，增加了约 **49.4%**。
   - 参数量增加的主要原因是解码器层每次融合了所有编码器层的特征。

2. **内存占用变化**：
   - **输入大小** 和 **前向/反向过程大小** 两者一致，表明两种网络在输入和特征处理上的对称性保持不变。
   - **参数大小** 增加了 `45.06 - 30.15 = 14.91 MB`，与参数量增加成正比。


---

## 怎么做特征共享？


### **1. 模型结构对比**
- **编码器-解码器设计**：
  - `BasicUNet` 的每个解码器层只连接同级别的编码器特征。
  - `CrossBasicUNet` 的每个解码器层融合了所有编码器层的特征（跨层共享），更能利用不同尺度的特征。

- **特征图尺寸和通道对比**：
  - 可以通过绘制网络结构图展示两者特征通道数和尺寸的变化，例如：
    - **`BasicUNet` 的跳跃连接**：
      ```
      Encoder_1 ----> Decoder_1
      Encoder_2 ----> Decoder_2
      Encoder_3 ----> Decoder_3
      Encoder_4 ----> Decoder_4
      ```
    - **`CrossBasicUNet` 的跨层连接**：
      ```
      Encoder_1, Encoder_2, Encoder_3, Encoder_4 ----> Decoder_1
      Encoder_1, Encoder_2, Encoder_3, Encoder_4 ----> Decoder_2
      Encoder_1, Encoder_2, Encoder_3, Encoder_4 ----> Decoder_3
      Encoder_1, Encoder_2, Encoder_3, Encoder_4 ----> Decoder_4
      ```



In [4]:
import torch
import torch.nn.functional as F

# 模拟不同编码器特征
encoder_features = [
    torch.randn(1, 32, 64, 64),
    torch.randn(1, 64, 32, 32),
    torch.randn(1, 128, 16, 16),
    torch.randn(1, 256, 8, 8)
]

# 解码器目标尺寸
target_size = (64, 64)

# 调整所有编码器特征到目标尺寸
resized_features = [F.interpolate(f, size=target_size, mode='nearest') for f in encoder_features]

# 拼接特征
combined_features = torch.cat(resized_features, dim=1)
print("Combined features shape:", combined_features.shape)


Combined features shape: torch.Size([1, 480, 64, 64])
